In [4]:
%iam_role arn:aws:iam::458226961977:role/glue_exec_role
%region ap-northeast-2
%connections steadio-segment-con

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Current iam_role is None
iam_role has been set to arn:aws:iam::458226961977:role/glue_exec_role.
Previous region: None
Setting new region to: ap-northeast-2
Region is set to: ap-northeast-2
Connections to be included:
steadio-segment-con


In [0]:
import sys
from awsglue import DynamicFrame
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext

Authenticating with profile=default
glue_role_arn defined by user: arn:aws:iam::458226961977:role/glue_exec_role
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 5
Session ID: 232e2761-ae6e-4784-a9ac-81d5b673e304
Applying the following default arguments:
--glue_kernel_version 0.31
--enable-glue-datacatalog true
Waiting for session 232e2761-ae6e-4784-a9ac-81d5b673e304 to get into ready status...
Session 232e2761-ae6e-4784-a9ac-81d5b673e304 has been created




In [1]:
glue_context = GlueContext(SparkContext.getOrCreate())
job = Job(glue_context)

username = "admin_user"
database = "dev"
password = "Hg6nNbUe24ud6WwM"
url = "jdbc:redshift://steadio-segment.cdftxbqtzhu1.ap-northeast-2.redshift.amazonaws.com:5439/dev"

In [16]:
source_dlab_ods_streak_outreach = glue_context.create_dynamic_frame.from_options(
    connection_type = "redshift",
    connection_options = {
                            "url": url,
                            "database" : database,
                            "user" : username,
                            "password" : password,
                            "query":"""
                                    select
                                        b.id as id
                                    ,	p.id as pipeline_id
                                    ,	p.name as pipeline_name
                                    ,	split_part(
                                        json_extract_path_text(
                                            json_extract_array_element_text(assignedtosharingentries, 0), 'email'
                                        ), '@', 1) as rep_name
                                    ,	case when json_extract_path_text(b.fields,'1003') != '' 
                                        then cast(json_extract_path_text(b.fields,'1003') as int) end as category_id
                                    ,   cast(b.stagekey as int) as stage_type_id
                                    ,	decode(json_extract_path_text(b.fields,'1014'), 'true', true, 'false', false) as is_tumblbug
                                    ,	case when json_extract_path_text(b.fields,'1002') != '' 
                                        then cast(json_extract_path_text(b.fields,'1002') as int) end as leadsource_id
                                    ,	b.createddate as created_at
                                    ,   cast(current_timestamp + interval '9 hour' as timestamp) as batched_at
                                    from tumblbug_dw.streak.boxes as b
                                    inner join tumblbug_dw.streak.pipelines as p
                                    on b.pipelineid = p.id
                                    where p.id = 'agxzfm1haWxmb29nYWVyMwsSDE9yZ2FuaXphdGlvbiIMdHVtYmxidWcuY29tDAsSCFdvcmtmbG93GICAxoW9psQLDA'
                                    """,
                            "redshiftTmpDir" : "s3://steadio-glue-info/temporary/",
                            "connectionName":"steadio-segment-con-jdbc",
                            "aws_iam_role": "arn:aws:iam::458226961977:role/redshift-exec-role"
                            }
                        )

In [17]:
source_dlab_ods_streak_outreach.printSchema()

root
|-- id: string
|-- pipeline_id: string
|-- pipeline_name: string
|-- rep_name: string
|-- category_id: int
|-- stage_type_id: int
|-- is_tumblbug: boolean
|-- leadsource_id: int
|-- created_at: timestamp
|-- batched_at: timestamp


In [19]:
pre_action = "TRUNCATE TABLE dlab.temp_streak_job"
pre_dlab_temp_streak_job = glue_context.write_dynamic_frame.from_jdbc_conf(
    frame=source_dlab_ods_streak_outreach
    ,catalog_connection="steadio-segment-con"
    ,connection_options={
        "database": "dev",
        "dbtable": "dlab.temp_streak_job",
        "preactions": pre_action
    }
    ,redshift_tmp_dir="s3://steadio-glue-info/temporary/"
)